In [1]:
import boto3
import pandas as pd
import numpy as np
import pickle
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import pandas_profiling
import os
from datetime import datetime, timedelta
import dateutil.relativedelta
import gc

pd.set_option('display.max_columns',None)

In [ ]:
import s3fs
import pyarrow.parquet as pq

def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe

In [ ]:
def _key_files__list(client, bucket, key):
    """return the key's size if it exist, else None"""
    response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    return response.get("Contents", [])

def get_s3_files(s3_url):
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    key = s3_url.split("/")[3:]
    key = "/".join(key)
    objects = _key_files__list(s3, bucket, key)
    files = ["s3://" + bucket + "/" + file["Key"] for file in objects]
    return files


def get_s3_parquet(s3_url, flag):
    """
    s3_url: path donde se desea buscar
    flag: permite identificar por cual camino entrar: 1: buscar archivo especifico, 0: buscar carpeta, ejemplo las std
    """
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    print('bucket_last_file: ', bucket)
    key = s3_url.split("/")[3:]
    key = "/".join(key)
    print('key_last_file: ', key)
#     if flag == "1":
#         objects = _key_files__list(s3, bucket, key)
#         #files = ["s3://" + bucket + "/" + file["Key"] for file in objects]
#         files = ["s3://" + bucket + "/" + file["Key"][:-9] for file in objects if
#                  file["Key"].split("/")[-1] == "_SUCCESS"]
#     elif flag == "0":
#         objects = _key_files__list(s3, bucket, key)
#         files = ["s3://" + bucket + "/" + file["Prefix"] for file in objects]
#     else:
#         files = []
    files = _key_files__list(s3, bucket, key)
    return files

In [ ]:
path = 's3://data-bpop-dev-refined/gestion-campanas/preaprobado-tc/'
bucket = 'data-bpop-dev-sandbox'